# GAN

In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
df1=pd.read_csv('fetureselectedcreditcard.csv')
Target=pd.read_csv('Target.csv')

lof_res=pd.read_csv('lof_res.csv')
lof_t=pd.read_csv('lof_t.csv')
lof_out=pd.read_csv('lof_out.csv')
lof_ot=pd.read_csv('lof_ot.csv')

isf_res=pd.read_csv('isf_res.csv')
isf_t=pd.read_csv('isf_t.csv')
isf_out=pd.read_csv('isf_out.csv')
isf_ot=pd.read_csv('isf_ot.csv')

ee_res=pd.read_csv('ee_res.csv')
ee_t=pd.read_csv('ee_t.csv')
ee_out=pd.read_csv('ee_out.csv')
ee_ot=pd.read_csv('ee_ot.csv')


osvm_res=pd.read_csv('osvm_res.csv')
osvm_t=pd.read_csv('osvm_t.csv')
osvm_out=pd.read_csv('osvm_out.csv')
osvm_ot=pd.read_csv('osvm_ot.csv')

In [3]:
lof_out=lof_out.to_numpy()
lof_ot=lof_ot.to_numpy()

isf_out=isf_out.to_numpy()
isf_ot=isf_ot.to_numpy()

ee_out=ee_out.to_numpy()
ee_ot=ee_ot.to_numpy()

osvm_out=osvm_out.to_numpy()
osvm_ot=osvm_ot.to_numpy()

In [4]:
sampling_res=pd.DataFrame(columns=['name','Train shape','normal percent','fraud percent','accuracy','precision','recall','f1-score','tn', 'fp', 'fn', 'tp'])
sampling_c=0

In [5]:
def generator(z, reuse=None):
    with tf.variable_scope("gen", reuse=reuse):
        hidden1 = tf.layers.dense(inputs=z, units=128)
        alpha = 0.01
        hidden1 = tf.maximum(alpha*hidden1, hidden1)
        hidden2 = tf.layers.dense(inputs=hidden1, units=64)
        hidden2 = tf.maximum(alpha*hidden2, hidden2)
        output = tf.layers.dense(hidden2, units=18, activation=tf.nn.tanh)
        return output

In [6]:
def discriminator(X, reuse=None):
    with tf.variable_scope("dis", reuse=reuse):
        hidden1 = tf.layers.dense(inputs=X, units=128)
        alpha = 0.01
        hidden1 = tf.maximum(alpha*hidden1, hidden1)
        hidden2 = tf.layers.dense(inputs=hidden1, units=128)
        hidden2 = tf.maximum(alpha*hidden2, hidden2)
        logits = tf.layers.dense(hidden2, units=1)
        output = tf.sigmoid(logits)
        return output, logits

In [7]:
real_data = tf.placeholder(tf.float32, shape=[None, 18])
z = tf.placeholder(tf.float32, shape=[None, 100])

In [8]:
G = generator(z)

In [9]:
D_output_real, D_logits_real = discriminator(real_data)
D_output_fake, D_logits_fake  = discriminator(G, reuse=True)

In [10]:
# LOSS
def loss_func(logits_in, labels_in):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits_in, labels=labels_in))

In [11]:
D_real_loss = loss_func(D_logits_real, tf.ones_like(D_logits_real)*0.9)
D_fake_loss = loss_func(D_logits_fake, tf.zeros_like(D_logits_real))

In [12]:
D_loss = D_real_loss + D_fake_loss
G_loss = loss_func(D_logits_fake, tf.ones_like(D_logits_fake))

In [13]:
learning_rate = 0.001
tvars = tf.trainable_variables()

d_vars = [var for var in tvars if 'dis' in var.name]
g_vars = [var for var in tvars if 'gen' in var.name]

In [14]:
D_trainer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(D_loss, var_list=d_vars)
G_trainer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(G_loss, var_list=g_vars)

In [15]:
def next_batch(num, data):
    '''
    Return a total of `num` random samples. 
    '''
    idx = np.arange(0, len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data.iloc[i] for i in idx]

    return np.asarray(data_shuffle)

In [16]:
# def resmaker1(name,X_resampled, y_resampled,sampling_c,X_test,y_test):
#     RF = RandomForestClassifier()
#     RF.fit(X_resampled, y_resampled)
#     preds = RF.predict(X_test)
#     y_resampled=y_resampled.reshape(-1,)
#     y_resampled = pd.Series(y_resampled)
#     classes=y_resampled.value_counts()
#     normal_share=round(classes[0]/y_resampled.count()*100,2)
#     fraud_share=round(classes[1]/y_resampled.count()*100, 2)
#     R=classification_report(y_test, preds,output_dict=True)
#     sampling_res.loc[sampling_c]=[name,X_resampled.shape,normal_share,fraud_share,R['accuracy'],R['macro avg']['precision'],R['macro avg']['recall'],R['macro avg']['f1-score']]
#     sampling_c+=1
#     return sampling_c

In [17]:
k=296

In [18]:
def resmaker1(k,name,X_resampled, y_resampled,sampling_c,X_test,y_test):
#     RF = RandomForestClassifier()
#     RF.fit(X_resampled, y_resampled)
#     preds = RF.predict(X_test)
#     classes=y_resampled.value_counts()
#     normal_share=round(classes[0]/y_resampled.count()*100,2)
#     fraud_share=round(classes[1]/y_resampled.count()*100, 2)
#     R=classification_report(y_test, preds,output_dict=True)
#     tn, fp, fn, tp = confusion_matrix(y_test, preds).ravel()
#     sampling_res.loc[sampling_c]=[name,X_resampled.shape,normal_share.Class,fraud_share.Class,R['accuracy'],R['macro avg']['precision'],R['macro avg']['recall'],R['macro avg']['f1-score'],tn, fp, fn, tp]
#     sampling_c+=1
#     return sampling_c
    xdfname=' X_resampled '+' + '+name+'.csv'
    ydfname=' y_resampled '+' + '+name+'.csv'
    X_resampled.to_csv(str(k)+xdfname,index=False)
    y_resampled.to_csv(str(k)+ydfname,index=False)
    X_test.to_csv(str(k)+' test '+xdfname,index=False)
    y_test.to_csv(str(k)+' test '+ydfname,index=False)
    return 1

In [19]:
def GAN(X_train,y_train,no_of_samples_to_generate):
    X_ones = X_train[y_train.Class == 1]
    X_ones.reset_index(drop=True, inplace=True)

    batch_size = 10
    epochs = 5000
    init = tf.global_variables_initializer()
    samples = []

    with tf.Session() as sess:
        sess.run(init)

        for epoch in range(epochs):

            num_batches = len(X_ones) // batch_size
            for i in range(num_batches):

                batch_X = next_batch(batch_size, X_ones)            
                batch_z = np.random.uniform(-1,1, size=(batch_size,100))

                _ = sess.run(D_trainer, feed_dict={real_data:batch_X, z: batch_z})
                _ = sess.run(G_trainer, feed_dict={z: batch_z})         

        sample_z = np.random.uniform(-1,1,size=(no_of_samples_to_generate, 100)) # new samples will be generated
        gen_sample = sess.run(generator(z, reuse=True), feed_dict={z:sample_z})
        samples.append(gen_sample)
    X_fake = pd.DataFrame(samples[0], columns=df1.columns)
    X_augmented_data = np.append(X_train, np.array(X_fake), axis=0)
    y_fake = np.ones(no_of_samples_to_generate)
    y_fake=y_fake.reshape(-1,1)
    y_augmented_data = np.append(y_train, y_fake, axis=0)
    return X_augmented_data,y_augmented_data

In [20]:
X_train, X_test, y_train, y_test = train_test_split(df1, Target, test_size = 0.3,random_state=0)
for i in [10129,21768,24542,39458,49409,66014,99164,198674]:
    X_resampled, y_resampled = GAN(X_train,y_train,no_of_samples_to_generate=i)
    X_resampled = pd.DataFrame (X_resampled,columns =X_train.columns)
    y_resampled = pd.DataFrame (y_resampled,columns =y_train.columns)
    sampling_c=resmaker1(k,'Gan',X_resampled, y_resampled,sampling_c,X_test,y_test)
    k=k+1

In [21]:
X_train, X_test, y_train, y_test = train_test_split(lof_res, lof_t, test_size = 0.3,random_state=0)
for i in [10129,21768,24542,39458,49409,66014,99164,198674]:
    X_resampled, y_resampled = GAN(X_train,y_train,no_of_samples_to_generate=i)
    X_resampled=np.concatenate((lof_out, X_resampled), axis=0)
    y_resampled=np.concatenate((lof_ot, y_resampled), axis=0)
    y_resampled[y_resampled==-1]=1
    X_resampled = pd.DataFrame (X_resampled,columns =X_train.columns)
    y_resampled = pd.DataFrame (y_resampled,columns =y_train.columns)
    sampling_c=resmaker1(k,'Gan + lof',X_resampled, y_resampled,sampling_c,X_test,y_test)
    k=k+1

In [22]:
X_train, X_test, y_train, y_test = train_test_split(isf_res, isf_t, test_size = 0.3,random_state=0)
for i in [10129,21768,24542,39458,49409,66014,99164,198674]:
    X_resampled, y_resampled = GAN(X_train,y_train,no_of_samples_to_generate=i)
    X_resampled=np.concatenate((isf_out, X_resampled), axis=0)
    y_resampled=np.concatenate((isf_ot, y_resampled), axis=0)
    y_resampled[y_resampled==-1]=1
    X_resampled = pd.DataFrame (X_resampled,columns =X_train.columns)
    y_resampled = pd.DataFrame (y_resampled,columns =y_train.columns)
    sampling_c=resmaker1(k,'Gan + isf',X_resampled, y_resampled,sampling_c,X_test,y_test)
    k=k+1

In [23]:
X_train, X_test, y_train, y_test = train_test_split(ee_res, ee_t, test_size = 0.3,random_state=0)
for i in [10129,21768,24542,39458,49409,66014,99164,198674]:
    X_resampled, y_resampled = GAN(X_train,y_train,no_of_samples_to_generate=i)
    X_resampled=np.concatenate((ee_out, X_resampled), axis=0)
    y_resampled=np.concatenate((ee_ot, y_resampled), axis=0)
    y_resampled[y_resampled==-1]=1
    X_resampled = pd.DataFrame (X_resampled,columns =X_train.columns)
    y_resampled = pd.DataFrame (y_resampled,columns =y_train.columns)
    sampling_c=resmaker1(k,'Gan + EllipticEnvelope',X_resampled, y_resampled,sampling_c,X_test,y_test)
    k=k+1

In [24]:
X_train, X_test, y_train, y_test = train_test_split(osvm_res, osvm_t, test_size = 0.3,random_state=0)
for i in [10129,21768,24542,39458,49409,66014,99164,198674]:
    X_resampled, y_resampled = GAN(X_train,y_train,no_of_samples_to_generate=i)
    X_resampled=np.concatenate((osvm_out, X_resampled), axis=0)
    y_resampled=np.concatenate((osvm_ot, y_resampled), axis=0)
    y_resampled[y_resampled==-1]=1
    X_resampled = pd.DataFrame (X_resampled,columns =X_train.columns)
    y_resampled = pd.DataFrame (y_resampled,columns =y_train.columns)
    sampling_c=resmaker1(k,'Gan + ocsvm',X_resampled, y_resampled,sampling_c,X_test,y_test)
    k=k+1